In [ ]:
import socket
import json

# PHD2 server connection details
HOST = 'localhost'
PORT = 4400

def get_guiding_stats():
    """Request and parse guiding stats from PHD2."""
    try:
        with socket.create_connection((HOST, PORT)) as sock:
            # Send a request for guiding stats
            request = json.dumps({"method": "get_stats", "id": 1}) + '\n'
            sock.sendall(request.encode('utf-8'))
            
            buffer = ""
            while True:
                # Receive data from the server
                data = sock.recv(4096).decode('utf-8')
                buffer += data

                # Split the buffer into complete JSON objects
                while "\n" in buffer:
                    line, buffer = buffer.split("\n", 1)
                    try:
                        # Parse each JSON object
                        response = json.loads(line)

                        # Check if the response contains guiding stats
                        if 'result' in response:
                            stats = response['result']
                            print(f"Star Position: X = {stats['avg_dist_x']}, Y = {stats['avg_dist_y']}")
                            return stats['avg_dist_x'], stats['avg_dist_y']
                        elif 'Event' in response:
                            print(f"Event received: {response['Event']} - Ignored")
                        else:
                            print(f"Unexpected response format: {response}")
                    except json.JSONDecodeError as e:
                        print(f"JSON decode error: {e}")
    except Exception as e:
        print(f"Error: {e}")
        return None, None

if __name__ == "__main__":
    x, y = get_guiding_stats()
    if x is not None and y is not None:
        print(f"Retrieved star position: X = {x}, Y = {y}")
    else:
        print("Failed to retrieve star position.")
